In [1]:
import praw
from praw.exceptions import APIException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
import requests
import time
from botlogger import createLogFile, logwrite

In [2]:
r = praw.Reddit("bot1")

In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')        

In [4]:
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get('https://tineye.com')

In [6]:
def comment(submission, matches, postList, submissionID, logMessage):
    print("hi i'm in the comment function now")
    try:
        submission.reply("There are " + 
               str(matches) + " pictures of this pet on the internet. (unless your pet is internet famous b/c in that case" +
                        " please introduce me to them ;) )")
        postList.append(submissionID)
        logwrite(logMessage)
        print("successful!")
    except APIException as err:
        time.sleep(360) #last time, API throttled by requiring a wait-time of 6 minutes
        logwrite("Hey, you got an APIException! Recursively calling the comment function")
        print("uh oh, recursion is happening")
        comment(submission, matches, postList, submissionID, "You got a APIException but you commented on this post!")

In [10]:
def reverseSearch(subredditName):
    createLogFile()
    r = praw.Reddit('bot1')
    subreddit=r.subreddit(subredditName)
    urls = []
    logwrite("Crawling through " + subredditName)
    
    for submission in subreddit.hot(limit=10):
        req = requests.get(submission.url)
        if 'image' in req.headers['content-type']: #and submission.author.name == 'uniquepic-bot': #checks to make sure it's an image
            urls.append((submission.id, submission.url))
#             print("got the right post!")
            
    logwrite("Checked 10 submissions. There are " + str(len(urls)) + " images")
#     fName = 'responded_posts'+subredditName+'.txt' #file contains ID's of submission posts that the bot has already commented on
    responded_posts = []
#     if os.path.isfile(fName):
#         with open(fName) as f:
#             content = f.readlines()
#         responded_posts = [line.strip() for line in content]
    
    for url in urls:
        print(url)
        url_box = driver.find_element_by_id('url_box')
        url_box.send_keys(url[1])
        url_box.submit()
        try:
            element_present = EC.presence_of_element_located((By.TAG_NAME, 'h2'))
            WebDriverWait(driver, 5).until(element_present) #waits 5 seconds to throw TimeoutException unless it finds element to return 
            results = driver.find_element_by_tag_name('h2') #result usually looks like "# Results"
            print(results.text.lower())
            if 'result' not in results.text.lower():
                print("hi")
                logwrite('Used up tineye searching limits. Quitting program')
                sys.exit()
            matches = int(results.text.split(' ')[0])
            logwrite('got a match - here it is: ' + str(matches))
            print('got a match - here it is:' + str(matches))
            if matches > 4 and url[0] not in responded_posts:
                print("this isn't unique!")
                logwrite("Found non-unique picture! There are " + str(matches) + " matches")
#                 submission = r.submission(id=url[0])
#                 comment(submission, matches, responded_posts, url[0], "A comment has been posted!")
        except TimeoutException as err:
            print("uh oh, TimeoutException")
            logwrite(err)
        except NoSuchElementException as err:
            print("uh oh, NoSuchElementException")
            logwrite(err)
        
        time.sleep(3)
        
#     with open(fName, 'w') as f:
#         print('about to write in a file!')
#         for post in responded_posts:
#             f.write(post + '\n')
    print('all done')

In [11]:
reverseSearch("dogpictures") #tried catpics, delightfullychubby, catpictures, dogpictures

('6uh397', 'http://i.imgur.com/mTa6K7T.jpg')
12 results
got a match - here it is:12
this isn't unique!
('6uhpp1', 'http://i.imgur.com/A4gsNSt.jpg')
you've reached your daily search limit.
hi


NameError: name 'sys' is not defined

Old things that I was testing (IGNORE EVERYTHING BELOW)

In [113]:
subreddit = r.subreddit("aww")
urls = []
for submission in subreddit.hot(limit=20):
    ending = submission.url.split('.')[-1]
    if ending == 'jpg' or ending == 'png':
        urls.append(submission.url)

In [114]:
urls

[('6u6xc2',
  'https://www.royalcanin.com/~/media/Royal-Canin/Product-Categories/cat-senior-landing-hero.ashx')]

In [74]:
import urllib.request as urllib

In [75]:
response = urllib.urlopen('https://images.google.com/searchbyimage?image_url=' + urls[0])
html = response.read()

In [60]:
from html.parser import HTMLParser

In [80]:
class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print ("Encountered a start tag:", tag)

    def handle_endtag(self, tag):
        print ("Encountered an end tag :", tag)

    def handle_data(self, data):
        print ("Encountered some data  :", data)

# instantiate the parser and fed it some HTML
parser = MyHTMLParser()
parser.feed(str(html))

Encountered some data  : b'
Encountered a start tag: html
Encountered a start tag: head
Encountered a start tag: meta
Encountered a start tag: meta
Encountered a start tag: meta
Encountered a start tag: meta
Encountered a start tag: title
Encountered some data  : Google
Encountered an end tag : title
Encountered a start tag: script
Encountered some data  : (function(){window.google={kEI:\'BciUWYiVFeHujwSC44_gDA\',kEXPI:\'1353382,1354276,1354401,1354443,1354514,1354625,1354750,1354946,1354987,3700307,3700347,3700442,4029815,4031109,4043492,4045841,4048347,4061945,4063220,4072775,4076999,4078430,4081039,4081164,4094544,4095910,4097153,4097470,4097922,4097929,4097951,4098733,4098740,4098752,4102238,4102827,4103475,4103861,4104258,4105115,4105128,4107555,4109316,4109490,4110656,4111276,4113148,4113217,4113275,4115290,4115338,4115697,4116724,4116731,4116926,4116927,4116935,4117980,4118103,4118227,4118438,4118626,4118798,4119025,4119032,4119034,4119036,4119239,4119272,4119656,4120293,4120294

In [124]:
testing = ['https://i.redd.it/q5foa3h2m4gz.jpg', 'https://gfycat.com/AcidicFluidAgama', 'https://i.imgur.com/jxOIJv4.gifv', 'https://i.redd.it/knbs8h5s43gz.jpg']

In [148]:
import time
import requests

In [152]:
driver = webdriver.Chrome()
driver.get("https://tineye.com/")
for test in testing:
    r = requests.get(test)
    if 'image' in r.headers['content-type']:
        url_box = driver.find_element_by_id('url_box')
        url_box.send_keys(test)
        url_box.submit()
        try:
            element_present = EC.presence_of_element_located((By.TAG_NAME, 'h2'))
            WebDriverWait(driver, 5).until(element_present)
            results = driver.find_element_by_tag_name('h2')
            matches = int(results.text.split(' ')[0])
            print(matches)
        except TimeoutException as err:
            print("It took too long to load!")
        except NoSuchElementException as err:
            print("No such element - sorry!")
        except StaleElementReferenceException as err:
            print("wtf")

        
#another problem is they're automatically selecting a picture of the gif for some reason??



64
0


In [154]:
import os

In [155]:
if os.path.isfile('testing.txt'):
    with open('testing.txt') as f:
        content = f.readlines()
    responded_posts = [line.strip() for line in content]


In [156]:
responded_posts

['123456', '234567', '345678', '456789']

In [158]:
responded_posts.append('273845')
responded_posts.append('945876')
responded_posts.append('2039480')

In [159]:
with open('testing.txt', 'w') as f:
    for post in responded_posts:
        f.write(post + '\n')

In [163]:
chrome_options = Options()
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get('https://tineye.com')